<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-dataset" data-toc-modified-id="Create-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Create dataset</a></span><ul class="toc-item"><li><span><a href="#ROIs" data-toc-modified-id="ROIs-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>ROIs</a></span><ul class="toc-item"><li><span><a href="#ROI-in-signal-dimension" data-toc-modified-id="ROI-in-signal-dimension-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>ROI in signal dimension</a></span></li></ul></li><li><span><a href="#ROI-in-navigation-dimension" data-toc-modified-id="ROI-in-navigation-dimension-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>ROI in navigation dimension</a></span></li><li><span><a href="#Decomposition" data-toc-modified-id="Decomposition-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Decomposition</a></span></li></ul></li><li><span><a href="#Curve-fitting" data-toc-modified-id="Curve-fitting-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Curve fitting</a></span></li></ul></div>

# Create dataset

For this tutorial we are going to start by creating a dataset that does not fit in the RAM. In particular a spectrum image of size 

In [1]:
%matplotlib qt

In [2]:
import hyperspy.api as hs

Let's import several libraries that we'll need to create the dataset

In [3]:
import skimage.data
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da

In [6]:
def size_in_GB(shape, dtype):
    """Print the size of an array in GB
    
    Parameters
    ----------
    shape: tuple
        Shape of the array
    dtype: string or numpy dtype
        Valid numpy data type
        
    """
    size = np.prod(shape) * np.dtype(dtype).itemsize / 1e9
    print("%.3f GB" % size)

In [12]:
size_in_GB((128,)*3, dtype="float")

0.017 GB


First lets load 4 512x512 images from skimage, convert them to grey and stack them

In [13]:
camera = hs.signals.Signal2D(skimage.data.load("camera.png", as_grey=True))
astronaut = hs.signals.Signal2D(skimage.data.load("astronaut.png", as_grey=True))
moon = hs.signals.Signal2D(skimage.data.load("moon.png", as_grey=True))
ihc = hs.signals.Signal2D(skimage.data.load("ihc.png", as_grey=True))
moon.change_dtype("float64")
moon.data /= moon.data.max()
camera.change_dtype("float64")
camera.data /= camera.data.max()
imgs = hs.stack([camera, astronaut, moon, ihc])

/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/skimage/data/__init__.py:63: UserWarning: `as_grey` has been deprecated in favor of `as_gray` and will be removed in v0.16.
  warn('`as_grey` has been deprecated in favor of `as_gray`'
/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/skimage/data/__init__.py:63: UserWarning: `as_grey` has been deprecated in favor of `as_gray` and will be removed in v0.16.
  warn('`as_grey` has been deprecated in favor of `as_gray`'
/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/skimage/data/__init__.py:63: UserWarning: `as_grey` has been deprecated in favor of `as_gray` and will be removed in v0.16.
  warn('`as_grey` has been deprecated in favor of `as_gray`'
/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/skimage/data/__init__.py:63: UserWarning: `as_grey` has been deprecated in favor of `as_gray` and will be removed in v0.16.
  warn('`as_grey` has been deprecated in favor of `as_gray`'


In [6]:
imgs

<Signal2D, title: Stack of , dimensions: (4|512, 512)>

In [7]:
hs.plot.plot_images(imgs, per_row=2, axes_decor="off")

Now lets create 4 spectral signals composed of randomly distributed and weighted gaussians




In [14]:
spectra = []
spectra_c = []
sd = hs.signals.Signal1D(np.zeros((4096,)))
sd.axes_manager[-1].units = "eV"
sd.axes_manager[-1].name = "Energy"

for i in range(4):
    m = sd.create_model()
    for position in np.random.randint(0, 4096, size=np.random.randint(8, 20)):
        m.append(
            hs.model.components1D.GaussianHF(
                centre=position,
                fwhm=np.random.randint(10, 500),
                height=np.random.randint(1, 100)))
    spectra.append(m.as_signal())
    spectra_c.append(
        hs.model.components1D.ScalableFixedPattern(
            signal1D=spectra[-1], interpolate=False))
    spectra_c[-1].shift.free = False
    spectra_c[-1].xscale.free = False
    spectra_c[-1].name = str(i)
spectra = hs.stack(spectra)
ax = hs.plot.plot_spectra(spectra, style="cascade")

In [15]:
imgsr = imgs
# imgsr = imgs.rebin((4, 256, 256))
# imgsr = imgs.rebin((4, 64, 64))

Create a *lazy* spectrum image

In [16]:
SIs = []
for spectrum, image in zip(spectra, imgsr):
    iml = image.T
    iml.data = da.from_array(iml.data, chunks=(128, 128))
    spectrum.data = da.from_array(spectrum.data, chunks=(128))
    SIs.append(iml.as_lazy() * spectrum.as_lazy())

/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/dask/array/core.py:2539: UserWarning: Increasing number of chunks by factor of 16
  (nparts / max_parts))
/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/dask/array/core.py:2539: UserWarning: Increasing number of chunks by factor of 16
  (nparts / max_parts))
/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/dask/array/core.py:2539: UserWarning: Increasing number of chunks by factor of 16
  (nparts / max_parts))
/home/francisco/anaconda/anaconda/lib/python3.6/site-packages/dask/array/core.py:2539: UserWarning: Increasing number of chunks by factor of 16
  (nparts / max_parts))


In [17]:
s = SIs[0] + SIs[1] + SIs[2] + SIs[3] 

In [18]:
print(s.data.chunks)

((128, 128, 128, 128), (128, 128, 128, 128), (128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128))


Add *lazy* poisson noise

In [19]:
s.data = da.random.poisson(s.data, chunks=s.data.chunks)

In [20]:
%%time
s.save("lazy_demo_data_h5py_chunking", chunks=True)

INFO:hyperspy.io_plugins.hspy:Chunks used for saving: (16, 16, 128)
INFO:hyperspy.io:The lazy_demo_data_h5py_chunking.hspy file was created


CPU times: user 4min 56s, sys: 1min 31s, total: 6min 28s
Wall time: 2min 6s


In [21]:
%%time
s.save("lazy_demo_data_hyperspy_chunking")

INFO:hyperspy.io_plugins.hspy:Chunks used for saving: (4, 4, 4096)
INFO:hyperspy.io:The lazy_demo_data_hyperspy_chunking.hspy file was created


CPU times: user 4min 55s, sys: 1min 31s, total: 6min 27s
Wall time: 1min 56s


In [22]:
s.T.save("lazy_demo_data_hyperspy_image_chunking")

INFO:hyperspy.io_plugins.hspy:Chunks used for saving: (1, 512, 512)
INFO:hyperspy.io:The lazy_demo_data_hyperspy_image_chunking.hspy file was created


In [24]:
s.T.save("lazy_demo_data_h5py_image_chunking", chunks=True)

INFO:hyperspy.io_plugins.hspy:Chunks used for saving: (128, 16, 16)
INFO:hyperspy.io:The lazy_demo_data_h5py_image_chunking.hspy file was created


In [19]:
s.plot(navigator=None)